In [ ]:
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

import os
import cv2
from PIL import Image
import numpy as np

image_directory = ''
SIZE = 150
dataset = []  
label = []

# Process Supine Neck Lift images
supine_neck_lift_videos = os.listdir(image_directory + 'Supine_Neck_Lift/')
for video_name in supine_neck_lift_videos:
    frames_directory = image_directory + 'Supine_Neck_Lift/' + video_name + '/'
    for i, frame_name in enumerate(os.listdir(frames_directory)):
        if frame_name.endswith('.jpg'):
            optical_flow_image = cv2.imread(os.path.join(frames_directory, frame_name))
            optical_flow_image = Image.fromarray(optical_flow_image, 'RGB')
            optical_flow_image = optical_flow_image.resize((SIZE, SIZE))
            dataset.append(np.array(optical_flow_image))
            label.append('Supine_Neck_Lift')

# Process Quadruped Rotation images
quadruped_rotation_videos = os.listdir(image_directory + 'Quadruped_Rotation/')
for video_name in quadruped_rotation_videos:
    frames_directory = image_directory + 'Quadruped_Rotation/' + video_name + '/'
    for i, frame_name in enumerate(os.listdir(frames_directory)):
        if frame_name.endswith('.jpg'):
            optical_flow_image = cv2.imread(os.path.join(frames_directory, frame_name))
            optical_flow_image = Image.fromarray(optical_flow_image, 'RGB')
            optical_flow_image = optical_flow_image.resize((SIZE, SIZE))
            dataset.append(np.array(optical_flow_image))
            label.append('Quadruped_Rotation')

# Process Thoracic Rotations images
thoracic_rotations_videos = os.listdir(image_directory + 'Thoracic_Rotations/')
for video_name in thoracic_rotations_videos:
    frames_directory = image_directory + 'Thoracic_Rotations/' + video_name + '/'
    for i, frame_name in enumerate(os.listdir(frames_directory)):
        if frame_name.endswith('.jpg'):
            optical_flow_image = cv2.imread(os.path.join(frames_directory, frame_name))
            optical_flow_image = Image.fromarray(optical_flow_image, 'RGB')
            optical_flow_image = optical_flow_image.resize((SIZE, SIZE))
            dataset.append(np.array(optical_flow_image))
            label.append('Thoracic_Rotations')

dataset = np.array(dataset)
label = np.array(label)
print("Dataset size is ", dataset.shape)
print("Label size is ", label.shape)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset, label, test_size = 0.20, random_state = 0)
print("Train size is ", X_train.shape)
print("Test size is ", X_test.shape)

from keras.utils import normalize
X_train = normalize(X_train, axis=1)
X_test = normalize(X_test, axis=1)



#Create/Define the model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

INPUT_SHAPE = (SIZE, SIZE, 3)

model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=INPUT_SHAPE))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), kernel_initializer='he_uniform'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128))  
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(3)) 
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',  
              optimizer='rmsprop', 
              metrics=['accuracy'])

print(model.summary())


history = model.fit(X_train, 
                    y_train, 
                    batch_size=64, 
                    verbose=1, 
                    epochs=100, 
                    validation_data=(X_test, y_test),
                    shuffle=False
                    )

model.save('models/exercise_recognition_model.h5')

#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
